<a href="https://colab.research.google.com/github/eyaler/workshop/blob/master/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BERT demo notebook
###by Eyal Gruss
###Hebrew support: Doron Adler
###⭐ New: Hebrew poetry glitcher - ShirBert! ⭐
###Based on https://huggingface.co/transformers
<img src='https://i.pinimg.com/originals/1a/38/8d/1a388d9b1e1ce42f424e60ce5b9d88ff.png' width="400px"/>

###Image credit: Doron Adler



In [ ]:
pip install transformers

In [ ]:
import numpy as np
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import re

def run_model(text, add_special_tokens=True, embedding=False, use_cls=False):
  # Tokenize input
  tokenized_text = tokenizer.tokenize(text)
  #print(tokenized_text)

  # Convert token to vocabulary indices
  indexed_tokens = tokenizer.encode(text, add_special_tokens=add_special_tokens or use_cls)

  # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
  segments_ids = [0]*len(indexed_tokens)

  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  # If you have a GPU, put everything on cuda
  tokens_tensor = tokens_tensor.to('cuda')
  segments_tensors = segments_tensors.to('cuda')

  if not embedding:
    # Predict all tokens
    with torch.no_grad():
        outputs = masked_model(tokens_tensor, token_type_ids=segments_tensors)
    if add_special_tokens:
      return indexed_tokens[1:-1], outputs[0][0][1:-1]  
    return indexed_tokens, outputs[0][0]

  with torch.no_grad():
      outputs = bert_model(tokens_tensor, token_type_ids=segments_tensors)
  encoded_layers = outputs.last_hidden_state[0].cpu()
  if use_cls:
    return encoded_layers[0] 
  return encoded_layers.mean(axis=0)

def predict_missing_word(text, topn=10, add_special_tokens=True):
  indexed_tokens, predictions = run_model(text, add_special_tokens=add_special_tokens)
  
  # Mask a token that we will try to predict back with `BertForMaskedLM`
  masked_index = indexed_tokens.index(tokenizer.convert_tokens_to_ids('[MASK]'))

  predicted_inds = torch.argsort(-predictions[masked_index])
  predicted_probs = [round(p.item(), 4) for p in torch.softmax(predictions[masked_index], 0)[predicted_inds]]
  predicted_tokens = tokenizer.convert_ids_to_tokens([ind.item() for ind in predicted_inds])
  return list(zip(predicted_tokens, predicted_probs))[:topn]

def complete_missing_word(text, add_special_tokens=True):
  word = predict_missing_word(text, topn=1, add_special_tokens=add_special_tokens)[0][0]
  return text.replace('[MASK]', word)

def join_clean(tokens):
  text = ' '.join(tokens)
  text = text.replace('ך ##', 'כ').replace('ם ##', 'מ').replace('נ ##', 'ן').replace('ף ##', 'פ').replace('ץ ##', 'צ').replace(' ##', '')
  text = text.replace(' ##', '').strip()
  text = tokenizer.clean_up_tokenization(text)
  text = text.replace(' :', ':').replace(' ;', ';').replace(' )', ')').replace('( ', '(')
  return text

def get_word_probs(text, add_special_tokens=True, with_masking=False):
  tokenized_text = tokenizer.tokenize(text)
  if with_masking:
    indexed_tokens = tokenizer.encode(text, add_special_tokens=False)
    predicted_probs = []
    for i in range(len(tokenized_text)):
      masked_tokens = tokenized_text[:]
      masked_tokens[i] = '[MASK]'
      text = join_clean(masked_tokens)
      _, predictions = run_model(text, add_special_tokens=add_special_tokens)
      predicted_probs.append(round(torch.softmax(predictions[i], 0)[indexed_tokens[i]].item(), 4))
  else:
    indexed_tokens, predictions = run_model(text, add_special_tokens=add_special_tokens)
    predicted_probs = [round(torch.softmax(predictions[i], 0)[j].item(), 4) for i,j in enumerate(indexed_tokens)]
  return list(zip(tokenized_text, predicted_probs))

def fix_hebrew_prefix(text):
  vav = 'ו'
  prefixes = 'מ|ש|ה|כ|ל|ב|כש'
  return re.sub('\\b('+vav+'?(?:'+prefixes+')|'+vav+') ', '\\1', text)

def is_hebrew_prefix(text):
  vav = 'ו'
  prefixes = 'מ|ש|ה|כ|ל|ב|כש'
  return text and re.fullmatch(vav+'?('+prefixes+')?', text)

def fix_one_word(text, add_special_tokens=True, with_masking=False, join_subwords=True, add_period=False, prevent_nonword=False, prefer_word=False, prevent_repeat=False, fix_hebrew=True):
  added = False
  if add_period and text[-1] not in '!?,.:;':
    added = True
    text += '.'
  tokenized_text = tokenizer.tokenize(text)
  if '[MASK]' in tokenized_text:
    ind = tokenized_text.index('[MASK]')
    bad_word = '[MASK]'
  else:
    probs = [p[1] for p in get_word_probs(text, add_special_tokens=add_special_tokens, with_masking=with_masking)]
    if added:
      probs[-1] = 1
    ind = torch.argmin(torch.tensor(probs))
    if join_subwords:
      while ind>0 and tokenized_text[ind].startswith('##'):
        ind -= 1
      bad_word = tokenized_text[ind]
      i = ind+1  
      while i<len(tokenized_text) and tokenized_text[i].startswith('##'):
        bad_word += ' '+tokenized_text[ind]
        del tokenized_text[i]
    else:
      bad_word = tokenized_text[ind]
    tokenized_text[ind] = '[MASK]'
    text = join_clean(tokenized_text)
  candidates = predict_missing_word(text, topn=None, add_special_tokens=add_special_tokens)
  fix = bad_word
  for word, _ in candidates:
    if word != bad_word and (not prevent_nonword or word!='[UNK]' and (re.search(r'\w(?<!(\d|_))',word) and (tokenized_text[ind][-1] not in 'ךםןףץ' or len(tokenized_text)==ind+1 or tokenized_text[ind+1].startswith('##')) or not prefer_word and bad_word and not re.search(r'\w(?<!(\d|_))',bad_word)) and (not is_hebrew_prefix(word) and (not re.fullmatch('[א-ת]',word) or not prefer_word and re.fullmatch('[א-ת]',bad_word)) or is_hebrew_prefix(word) and len(tokenized_text)>ind+1 and re.search(r'\w(?<!(\d|_))',tokenized_text[ind+1])) and (not word.startswith('##') or ind>0 and re.search(r'\w(?<!(\d|_))',tokenized_text[ind-1]))):
      fix = word
      if not prevent_repeat or word not in tokenized_text or len(word)==1:
        break
  text = text.replace('[MASK]', fix)
  text = join_clean([text])
  if fix_hebrew:
    text = fix_hebrew_prefix(text)
  if added:
    text = text[:-1]
  return text

def cosim(vec1, vec2):
  return np.dot(vec1,vec2)/np.linalg.norm(vec1)/np.linalg.norm(vec2)

def sent_sim(base_sent, compare_to, add_special_tokens=True, use_cls=False):
  results = []
  if type(compare_to)==str:
    compare_to = [compare_to]
  e1 = run_model(base_sent, add_special_tokens=add_special_tokens, embedding=True, use_cls=use_cls)
  for s in compare_to:
    e2 = run_model(s, add_special_tokens=add_special_tokens, embedding=True, use_cls=use_cls)
    results.append(cosim(e1,e2))
  if len(results)==1:
    return results[0]
  return results

def mask_join(part1, part2, add_period=False):
  s = part1 + ' [MASK] ' + part2  
  if add_period and s[-1] not in '!?,.:;':
    s += '.'
  return s

In [ ]:
#@title Choose model { run: "auto" }

model = 'bert-base-uncased' #@param ['bert-base-uncased', 'bert-large-uncased', 'bert-large-uncased-whole-word-masking', 'bert-base-multilingual-cased']

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(model)

# Load pre-trained model weights and change to evaluation mode
masked_model = BertForMaskedLM.from_pretrained(model)
masked_model.eval()
masked_model.to('cuda')

bert_model = BertModel.from_pretrained(model)
bert_model.eval()
bert_model.to('cuda')

print('\nhttps://huggingface.co/'+model)


In [ ]:
predict_missing_word('The boy [MASK] to his school.')

In [ ]:
predict_missing_word('Alex likes to have [MASK] with his best friend.')

In [ ]:
get_word_probs('The boy want to his school.')

In [ ]:
get_word_probs('The boy want to his school.', with_masking=True)

In [ ]:
fix_one_word('The boy want to his school.')

In [ ]:
get_word_probs('The boy want to his school')

In [ ]:
get_word_probs('The boy want to his school', with_masking=True)

In [ ]:
get_word_probs('The boy returned to his school.')

In [ ]:
get_word_probs('The boy returned to his school.', with_masking=True)

In [ ]:
get_word_probs('he said')

In [ ]:
get_word_probs('he said', with_masking=True)

In [ ]:
get_word_probs('he said.')

In [ ]:
get_word_probs('he said.', with_masking=True)

In [ ]:
predict_missing_word('The prime minister [MASK]')

In [ ]:
predict_missing_word('The prime minister [MASK].') #added period in the end

In [ ]:
complete_missing_word('The prime minister [MASK].')

In [ ]:
get_word_probs('The crime minister resigned.')

In [ ]:
get_word_probs('The crime minister resigned.', with_masking=True)

In [ ]:
get_word_probs('. The crime minister resigned.') #add period in beginning 

In [ ]:
get_word_probs('. The crime minister resigned.', with_masking=True)

In [ ]:
fix_one_word('. The crime minister resigned.')

In [ ]:
base_sent = 'she told me she loved me before she passed away'
compare_to = [
              'he told me he loved me before she passed away',
              'he told me that you loved her before i passed away',
              'i was very sad when my love died',
              'you are my one and only love for eternity',
              'i love pizza more than i love sex',
              'we must have some pizza with onions',
              'sieg heil',
              'יאללה ביי'
              ]   
sorted(zip(sent_sim(base_sent, compare_to), compare_to),reverse=True)

In [ ]:
#@title Choose model { run: "auto" }

model = 'TurkuNLP/wikibert-base-he-cased' #@param ['TurkuNLP/wikibert-base-he-cased', 'bert-base-multilingual-cased']

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(model)

# Load pre-trained model weights and change to evaluation mode
masked_model = BertForMaskedLM.from_pretrained(model)
masked_model.eval()
masked_model.to('cuda')

bert_model = BertModel.from_pretrained(model)
bert_model.eval()
bert_model.to('cuda')

print('\nhttps://huggingface.co/'+model)

In [ ]:
s = 'ישראל [MASK] ולתפארת'
print(s+'\n')
predict_missing_word(s)

In [ ]:
s = 'ולתפארת [MASK] ישראל' #fixed order
print(s+'\n')
predict_missing_word(s)

In [ ]:
s = 'ולתפארת [MASK] ישראל' + '.' #added period
print(s+'\n')
predict_missing_word(s)

In [ ]:
get_word_probs('לא רוצה')

In [ ]:
get_word_probs('לא רוצה', with_masking=True)

In [ ]:
get_word_probs('לא רוצה.')

In [ ]:
get_word_probs('לא רוצה.', with_masking=True)

In [ ]:
#פרדי מרקורי מאסק זמר ומוזיקאי

p1 = 'פרדי מרקורי'
p2 = 'זמר ומוזיקאי'
s = mask_join(p1,p2,add_period=True)
print(s+'\n')
predict_missing_word(s)

In [ ]:
#פרדי מרקורי היה מאסק ומוזיקאי

p1 = 'פרדי מרקורי היה'
p2 = 'ומוזיקאי'
s = mask_join(p1,p2,add_period=True)
print(s+'\n')
predict_missing_word(s)

#שירברט - משבש (משברט) שירה עברית
#Hebrew poetry glitcher - ShirBert
###Best used with TurkuNLP/wikibert-base-he-cased model
###Warning: poetic experimentation below this line
<img src='https://i.pinimg.com/originals/e2/8d/08/e28d0819c670b164eceb38d7e5acb466.jpg' width="400px"/>

###Image credit: [thismuppetdoesnotexist.com](https://thismuppetdoesnotexist.com) by Doron Adler and EG


In [ ]:
def glitch_line(line, long_intersect=1, short_count=2, with_masking=False, add_period=False, prevent_nonword=True, prefer_word=True, prevent_repeat=False, fix_hebrew=False, verbose=False):
  hist = []
  tokens = set(re.findall(r"\b(?:\w'?){2,}\b", line))
  while line not in hist and len(tokens&set(re.findall(r"\b(?:\w'?){2,}\b", line)))>(long_intersect if len(tokens)>short_count else 0):
    hist.append(line)
    line = fix_one_word(line, with_masking=with_masking, add_period=add_period, prevent_nonword=prevent_nonword, prefer_word=prefer_word, prevent_repeat=prevent_repeat, fix_hebrew=fix_hebrew)
    if verbose:
      print('>'*verbose+line)
    if line in hist:
      line = hist[-1]
  return line

def shirbert(text, remove_nikud=True, split_eos=False, with_masking=False, add_period_to_short=False, prevent_nonword=True, prefer_word=True, prevent_repeat=False, verbose=False):
  output = ''
  min_sim = 1
  levels = set()
  if remove_nikud:
    text = re.sub('[\u0591-\u05bd\u05bf-\u05c2\u05c4\u05c5\u05c7]','',text) # remove all nikud and teamim except makaf, sof-pasuk, nun-hafukha
  if split_eos:
    text = text.replace('. ','.\n').replace('! ','!\n').replace('? ','?\n')
  for line in text.strip().splitlines():
    line = line.strip()
    orig_line = line
    short_tokens = re.findall(r"\b(?:\w'?){2,}\b", line)
    num_tokens = len(short_tokens)
    short_tokens = set(short_tokens)
    long_tokens = {t for t in short_tokens if len(t)>=3}
    add_period = num_tokens>2 or add_period_to_short and line[-1] not in '!?,.:;'
    line = glitch_line(line, with_masking=with_masking, add_period=add_period, prevent_nonword=prevent_nonword, prefer_word=prefer_word, prevent_repeat=prevent_repeat, verbose=1 if verbose else 0)
    sim = sent_sim(orig_line, line)
    min_sim = min(min_sim,sim)
    levels.add(1)
    if verbose:
      print(f'>sim={sim:.2f}')
    tokens = set(re.findall(r"\b(?:\w'?){2,}\b", line))
    if (len(long_tokens&tokens)>2 or len(short_tokens&tokens)>0 and len(short_tokens)<=2) and add_period:
      line = glitch_line(orig_line, with_masking=with_masking, prevent_nonword=prevent_nonword, prefer_word=prefer_word, prevent_repeat=prevent_repeat, verbose=2 if verbose else 0)
      sim = sent_sim(orig_line, line)
      min_sim = min(min_sim,sim)
      levels.add(2)
      if verbose:
        print(f'>>sim={sim:.2f}')
    tokens = set(re.findall(r"\b(?:\w'?){2,}\b", line))
    if len(long_tokens&tokens)>2 or len(short_tokens&tokens)>0 and len(short_tokens)<=2:
      mask_line = orig_line.replace(sorted(short_tokens,key=len)[-1], '[MASK]', 1)
      add_period = num_tokens>2 or add_period_to_short and mask_line[-1] not in '!?,.:;'
      line = glitch_line(mask_line, with_masking=with_masking, add_period=add_period, prevent_nonword=prevent_nonword, prefer_word=prefer_word, prevent_repeat=prevent_repeat, verbose=3 if verbose else 0)
      sim = sent_sim(orig_line, line)
      min_sim = min(min_sim,sim)
      levels.add(3)
      if verbose:
        print(f'>>>sim={sim:.2f}')
      tokens = set(re.findall(r"\b(?:\w'?){2,}\b", line))
      if len(short_tokens&tokens)>0 and len(short_tokens)<=2 and add_period:
        line = glitch_line(mask_line, with_masking=with_masking, prevent_nonword=prevent_nonword, prefer_word=prefer_word, prevent_repeat=prevent_repeat, verbose=4 if verbose else 0)
        sim = sent_sim(orig_line, line)
        min_sim = min(min_sim,sim)
        levels.add(4)
        if verbose:
          print(f'>>>>sim={sim:.2f}')
    line = fix_hebrew_prefix(line)
    print(line)
    output += line+'\n'
  if verbose:
    print(f'>>>min_sim={min_sim:.2f} levels={sorted(levels)}')
  return output

poem = shirbert('''
התקוה
נפתלי ה. אימבר

כל עוד בלבב פנימה
נפש יהודי הומיה,
ולפאתי מזרח קדימה
עין לציון צופיה,
עוד לא אבדה תקותנו,
התקוה בת שנות אלפים,
להיות עם חפשי בארצנו,
ארץ ציון וירושלים.
''')

In [ ]:
poem = shirbert('''
_מעילי הפשוט ופנס על הגשר
ליל הסתיו ושפתי הלחות מני גשם
כך ראית אותי ראשונה התזכור
והיה לי ברור כמו שתיים ושתיים
כי אהיה בשבילך כמו לחם ומים
וכאל מים ולחם אלי תחזו
''')

In [ ]:
poem = shirbert('''
מהפכה, מתהפכים כמו 69
היום כן משתלם הפשע
אני מסתובב ברחובות מחפש מהות
אבל הכל חוזר חביבי כמו תקליט שרוט
ואינפלציה כבר מיפלציה מכאיבה כמו אפילציה
מה זה אינטגרציה?! השיטה מניפולציה
ואתה מחכה שמשהו יפסיק ת'כאב
כמו כדור 9 מילימטר בתוך הלב
אנחנו מטרות נעות
אין שטרות - אין בעיות
שטויות, הבטחות במים נכתבות
חולמים בעיניים פתוחות - לרווחה
פאק משרד הרווחה - לחם, עבודה
תמונות, צרחות, סירנות ויריות
אפילו בשכונת התקווה אבדו כל התקוות
יורד על הברכיים ''אלי אנא''
הנה זה בא (מי בא?) זינזאנה!
''')

In [ ]:
poem = shirbert('''
שתדעו.
מאת אודיה רוזנק

אני ועוד מיליון מובטלים
רואים את התמונות שאתם מעלים לאינסטגרם,
שתדעו.
אני ועוד מיליון מובטלים רואים את עוגות הקצפת
הלבנות שלכם, את המקררים העמוסים
יוגורטים אפס אחוז שומן,
שתדעו.
אני ועוד מיליון מובטלים רואים אתכם מורחים חמאה
על חלות תוצרת בית מקמח כוסמין
ומוסיפים פרוסת סלמון,
שתדעו.
אני ועוד מיליון מובטלים בלענו שעונים מעוררים
טיק טק
טיק טק
טיק טק
אתם שומעים?
אני ועוד מיליון מובטלים מתכוננים לצאת
לרחובות, לשבור לכם את החלונות
לשרוף לכם את האסמים
לתלוש לכם את הפנים, ולגלות את המסכות.
שתדעו.
''')

In [ ]:
poem = shirbert('''
האגם הגדול.
מאת רועי צ. ארד
 
שוחה לבד בתוך האגם הגדול
שוחה על בטני באגם הגדול
שוחה על גבי באגם הגדול
על צדי שוחה באגם הגדול
מדוע איש לא מצטרף אלי באגם הגדול?
אין גדר סביב האגם הגדול
משתכשך בתוך אגם הגדול
צולל בתוך אגם הגדול
הדרך לדפוק את השיטה: האגם הגדול
הצטרפי אלי אל האגם הגדול
הצטרף אלי אל האגם הגדול
מדוע אני לבד באגם הגדול?
דבר לא מונע מכם לבוא אל האגם הגדול
למשל אתה הקורא,
אל נא תאמר "אני רק הקורא",
הפשל המכנס, השלך החזיה,
בוא עכשיו אל
האגם הגדול!
שחה עמוק בתוך האגם הגדול!
שחה מהר בתוך האגם הגדול!
שחה על גחונך בהאגם הגדול!
שחה על העורף בהאגם הגדול!
בוא עכשיו לכאן.
פעם היו כאן רבים באגם הגדול
אני היחיד שטבל באגם הגדול
אפשר לטבוע בהאגם הגדול
(אבל) אפשר למות מצחצוח יתר במרידול
אז בואו בואו בואו אל האגם הגדול
נצוף נצוף נצוף באגם הגדול
אין כאן מים, רק קול
נתחכך בתוך האגם הגדול
בשרכם יוטח בבשרי באגם הגדול
בוא עכשיו לכאן.
מדוע אני לבד בתוך האגם הגדול
מדוע אני לבד בתוך האגם הגדול
כי אני לבד בתוך האגם הגדול
כן, אני לבד באגם הגדול.
אני לבד לבד לבד באגם הגדול
לפעמים עם עוד כמה חברים
מדוע אינכם מבינים שהכי סבבי באגם הגדול
שהכי חינמי באגם הגדול
שזה המקום היחיד בעיירה בלי גדרות, האגם הגדול
ולא איזה אשד הפכפך, האגם הגדול
והוא לא ממש גדול, האגם הגדול
אפשר לשים אותו בבגאז' של פג'ו,
בתא מטבעות מעור שסק
בפנקסון סגול
האגם הגדול האגם הגדול האגם הגדול
הצטרפו אלי עכשיו לאגם הגדול
הצטרפתו עמי באגם הגדול
יש מקום לכולם באגם הגדול
יש מקום לכולן באגם הגדול
יש מקום לקולר באגם הגדול
האגם הגד גד גד גדול
האגם הגדול דול דול דול דול
בואו אל האגם הגדול
בואו אל האגם הגדול
למה אתם נכנסים אל תוך האגם הגדול רק
כשאני יוצא מהמים להתייבש?
''')

In [ ]:
poem = shirbert('''
שני ארזי

מיקרוס פר בנרה הסינדוק משלם
קולו דנצירסמי אוטונומאנטילן
שוקולרצח בעד דם לא לוחשת מיד!
הציבני לגוף אהיה אל אחד!
הצילני מדם וזוועות משיחין

סעודת רשעים בגבולות היומרין
לא תראה בי אח מוות משתנה וזהה
מה עסוק בי רום דע את גבולותיך בשן!
אל תנשך אל תרצח אל תגנוב מתחתי
לא תלויה בכוחו עבדך אשריי 

לא נתלה בעור מוות ודום עומק גופך
מה שלי ונגזר עת דפך באפך!
לא נשבר ורועד השימני בצל
אשתנה אתגלד בתנועות יד ואל
ובשמי אתרשם ואראה חיסורים

לא תרביץ כשתשמע חיבורים אסורים!
מה נגזר בעול מוות משתנה ונכנס
היונה התפשטה במוחו הדרדס
ואנס ותם רוח מתעקש לעוד דם
לא שלום אדוני לא ירצח אף פעם

לא ארשה שתרצח אדוני את דמי
מה יקרה אם תיקח נשרים מבשרי
והרצח ינגע באורות שארית
אנדרגראונד נס ובא מתקשר לאורית
מי זה כאן הפולש בשיחת הבירור

מה אציל מתבייש רוטב אש הסיפור
לא מקריח יודע בגד דמות שם צמוד
שערות ופשרות רקדנית על עמוד
לא נכנסת פואנטה בהרגל אפשרי
לא יוצא אל ומטא חידוש השני
''')

In [ ]:
poem = shirbert('''
גן נעול

מִי אַתָּה? מַדּוּעַ יָד מוּשֶׁטֶת
לֹא פּוֹגֶשֶׁת יַד אָחוֹת?
וְעֵינַיִם אַךְ תַּמְתֵּנָּה רֶגַע
וְהִנֵה שָׁפְלוּ כְּבָר נְבוֹכוֹת.

גַּן נָעוּל. לֹא שְׁבִיל אֵלָיו, לֹא דֶרֶךְ.
גַּן נָעוּל – אָדָם.
הַאֵלֵךְ לִי? אוֹ אַכֶּה בַּסֶּלַע
עַד זוֹב דָּם? 
''')

In [ ]:
# גרסה מאת דורון אדלר
poem = shirbert('''
רחל - גן נעול

מי אתה? מדוע יד מושטת?
לא פוגשת יד אחות
ועיניים, אך תמתנה רגע
והנה שפלו כבר נבוכות.

גן נעול,
לא שביל אליו, לא דרך.
גם נעול, אדם, גן נעול.

ההלך לי? 
או אכה בסלע?

עד זוב דם
''')

In [ ]:
poem=shirbert('''
רק על עצמי

רַק עַל עַצְמִי לְסַפֵּר יָדַעְתִּי.
צַר עוֹלָמִי כְּעוֹלַם נְמָלָה,
גַּם מַשָּׂאִי עָמַסְתִּי כָּמוֹהָ
רַב וְכָבֵד מִכְּתֵפִי הַדַּלָּה.

גַּם אֶת דַרְכִּי – כְּדַרְכָּהּ אֶל צַמֶּרֶת –
דֶּרֶך מַכְאוֹב וְדֶרֶךְ עָמָל,
יַד עֲנָקִים זְדוֹנָה וּבוֹטַחַת,
יַד מִתְבַּדַּחַת שָׂמָה לְאַל.

כָּל אָרְחוֹתַי הִלִּיז וְהִדְמִיע
פַּחַד טָמִיר מִיַּד עֲנָקִים.
לָמָּה קְרָאתֶם לִי, חוֹפֵי הַפֶּלֶא?
לָמָּה כְּזַבְתֶּם, אוֹרוֹת רְחוֹקִים? 
''')